In [1]:
from pathlib import Path
import _fixpath

ANYLABEL_DIR = Path("/workspaces/cv/.tmp/to-annotate-20251010_163139-no-orientation")  # Where you annotated new stuff
LAST_ANNOTATION_DIR = Path("/workspaces/cv/data/ml/annotated/202505-dolphins")
NEW_ANNOTATION_DIR = Path("/workspaces/cv/data/ml/annotated/202508-dolphins")
SURVEY_NAME = "2025-08-winter-survey"
SURVEY_TYPE = "action-aviation-multicamera"
FLIGHT = None

In [5]:
import json

with open(LAST_ANNOTATION_DIR / "all.json") as f:
    last = json.load(f)


In [6]:
from datetime import datetime
from PIL import Image
from img_utils import is_exif_oriented

date_added = datetime.now().isoformat()

imgs_to_copy = []
for img_path in ANYLABEL_DIR.glob("*.jpg"):
    new_img_id = len(last["images"])
    assert new_img_id not in [i["id"] for i in last["images"]]
    assert not is_exif_oriented(img_path), f"Image {img_path} has EXIF orientation - please fix that first"
    img = Image.open(img_path)
    imgs_to_copy.append(img_path)
    assert img.width > img.height
    last["images"].append(
        {
            "id": new_img_id,
            "file_name": img_path.name,
            "width": img.width,
            "height": img.height,
            "meta": {
                "survey_type": SURVEY_TYPE,
                "survey": SURVEY_NAME,
                "flight": FLIGHT,
                "date_added_to_dataset": date_added,
            },
        }
    )

    # Load annotation:
    annotation_path = img_path.with_suffix(".json")
    with open(annotation_path) as f:
        annotation = json.load(f)

    for shape in annotation["shapes"]:
        assert shape["label"] == "maui"
        x0y0, x1y1 = shape["points"]
        x0 = int(x0y0[0])
        y0 = int(x0y0[1])
        w = int(x1y1[0] - x0y0[0])
        h = int(x1y1[1] - x0y0[1])
        assert w > 0 and h > 0
        new_annotation_id = len(last["annotations"])
        assert new_annotation_id not in [i["id"] for i in last["annotations"]]
        last["annotations"].append(
            {"id": new_annotation_id, "image_id": new_img_id, "category_id": 0, "bbox": [x0, y0, w, h]}
        )

assert len(last["images"]) == len(set(i["id"] for i in last["images"]))
assert len(last["images"]) == len(set(i["image_id"] for i in last["annotations"]))
assert len(last["annotations"]) == len(set(i["id"] for i in last["annotations"]))

In [4]:
# Now save it
NEW_ANNOTATION_DIR.mkdir(exist_ok=True)
with open(NEW_ANNOTATION_DIR / "all.json", "w") as f:
    json.dump(last, f, indent=2)

In [7]:
import shutil

# Copy it over
imgdir = NEW_ANNOTATION_DIR / "all"
imgdir.mkdir(exist_ok=True, parents=True)

# Copy old ones:
for img in (LAST_ANNOTATION_DIR / "all").iterdir():
    shutil.copy(img, imgdir / img.name)

# Copy new ones - the raw image:
for img_path in imgs_to_copy:
    shutil.copy(img_path, imgdir / img_path.name)

# Check it all makes sense
assert len(list(imgdir.iterdir())) == len(last["images"])
for img in last["images"]:
    assert (imgdir / img["file_name"]).exists()

# Save the annotation
with open(NEW_ANNOTATION_DIR / "all.json", "w") as f:
    json.dump(last, f, indent=2)